In [41]:
from decimal import*
import math
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
# importing tk stuff for GUI
import tkinter as tk
from tkinter import *
from tkinter import filedialog
import requests
from PIL import Image, ImageTk
import matplotlib.backends.backend_tkagg
from matplotlib.backend_bases import key_press_handler
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.ticker as mticker
import sys
import os 
os.chdir(r"C:\Users\Aman Khatri\Desktop\Work\AAM\Backlash Info")

In [42]:
#Load saved dataset
dataindexer=0
testit = os.getcwd()+r"\aam lash2\test data\1581260162.txt"
####### Main calculation and plotting loop, iterating with new data as selected #####

In [40]:
def main():
    LASH = pd.read_csv(str(testit), header=4)
    LASH.columns = ['CWG', 'PIN', 'CCWG', 'CCWP']
    Enc = pd.read_csv(str(testit), header=None,skiprows=1,nrows=3)
    encdt=Enc.at[2,0] #time increment of data in seconds
    tpin=Enc.at[1,0] #Pinion Teeth
    tgear=Enc.at[1,1] #Gear Teeth
    lgain =Enc.at[0,2] #diameter gain for lash calcs
    offset=Enc.at[0,3] #calculation offset
    Lmin=Enc.at[2,2] #Minimum Backlash limit MM
    Lmax=Enc.at[2,3] #Maximum Backlash limit MM
    Atype=Enc.at[0,1]
    sernum=Enc.at[0,0]
    sernum=int(sernum)
    ########## Converting variables to floating decimals ######

    tpin=float(tpin) 
    tgear=float(tgear) 
    ratio=tgear/tpin
    rdia=Enc.at[2,1] #raw diameter from file
    rdia = float(rdia) 
    lgain=float(lgain)
    dia=rdia*lgain #calculating diameter
    gencts=Enc.at[1,2] #gear encoder counts per rev
    gencts = int(gencts)
    pencts=Enc.at[1,3] #pinion encoder cts per rev
    pencts = int(pencts)
    #print(testit)
    print(LASH)
    gstart=LASH.loc[0,'CWG']
    gend=LASH['CWG'][LASH.index[-1]]
    pref=LASH.loc[0,'PIN']
    pref2=LASH.loc[0,'CCWP']
    gref2=LASH.loc[0,'CCWG']
    plast=LASH['PIN'][LASH.index[-1]]
    plast2=LASH['CCWP'][LASH.index[-1]]
    glast2=LASH['CCWG'][LASH.index[-1]]
    ratioCW=(plast-pref)/(gend-gstart)
    ratioCCW=(plast2-pref2)/(glast2-gref2)

    ### Starting CCW TE calculation at a matching pinion to illustrate backlash when plotting ##########

    glast2C=glast2+(plast-plast2)/ratio
    CWteend=gend-(plast-pref)/ratio-gstart

    LASH['DEG']=(LASH['CWG']-gstart)
    LASH['RAD']=(LASH['DEG']*np.pi/180)
    #print(gstart,gend)
    #print(pref,pref2,gref2,plast,plast2,glast2,glast2C)
    #print(ratioCW,ratioCCW)
    #print(LASH)
    #### Gear TE in Counts Clockwise & Counterclockwise ######## 
    LASH['TECW']=(LASH['CWG']-((LASH['PIN']-pref)/(tgear/tpin)+gstart))
    LASH['CW TE microrad']=(LASH['TECW']*np.pi*1000000/180)
    LASH['TECCW']=(LASH['CCWG']-(glast2C-(plast-LASH['CCWP'])/(tgear/tpin))+(glast2C-gend)+CWteend)
    LASH['CCW TE microrad']=(LASH['TECCW']*np.pi*1000000/180)
    #print(LASH)
    ### "Raw" Backlash Calculations without gain and offset ##########
    LASH['PINCOMP']=(LASH['PIN']-LASH['CCWP'])
    LASH['GEARCOMP']=(LASH['PINCOMP']/(tgear/tpin))
    LASH['COMPGRCCW']=(LASH['GEARCOMP']+LASH['CCWG'])
    LASH['RAW Backlash MM']=((LASH['COMPGRCCW']-LASH['CWG'])/360*np.pi*rdia)
    #print(LASH)
    ### Backlash calculations including gain and offset in values #########
    LASH['Backlash MM']=((LASH['COMPGRCCW']-LASH['CWG'])/360*np.pi*dia+offset)
    #print(LASH)
    ##### Pinion velocity calculations based on time increment of data acquisition #######
    for i in range(1, len(LASH)):
        LASH.loc[i, 'PCWrpm'] = (LASH.loc[i, 'PIN'] - LASH.loc[i-1, 'PIN'])/360/encdt*60
    for i in range(1, len(LASH)):
        LASH.loc[i, 'PCCWrpm'] = (LASH.loc[i, 'CCWP'] - LASH.loc[i-1, 'CCWP'])/360/encdt*60
    LASH.loc[LASH['PCCWrpm']>150,'PCCWrpm']= 0.0
    LASH.loc[LASH['PCCWrpm']<10,'PCCWrpm']= 'NaN'
    #print(LASH)
    #LASH.to_csv('LASH-DATA.csv', sep=',')
    #### Parameters calculated to facilitate TE PLot limits and scaling of the plot ###########
    temaxcw=LASH['CW TE microrad'].max()
    temincw=LASH['CW TE microrad'].min()
    temaxccw=LASH['CCW TE microrad'].max()
    teminccw=LASH['CCW TE microrad'].min()
    if temaxcw > temaxccw:
        temax = temaxcw+100
    else:
        temax= temaxccw+100
    if temincw < teminccw:
        temin = temincw-100
    else:
        temin= teminccw-100
    teasp=(130/abs((temax-temin))) 

    temax=round(temax, -2)
    temin=round(temin, -2)
    teasp=abs(round(teasp, 2))

    def round_up(n):
        multiplier = .001
        return math.ceil(n*multiplier)/multiplier

    def round_dwn(n):
        multiplier = 0.001
        return math.floor(n*multiplier)/multiplier


    temaxl=round_up(temax)
    teminl=round_dwn(temin)
    teindx=int((temaxl-teminl)/1000+1)

    teconv=rdia/1000000.0

    print(temin,temax,teminl,temaxl,teindx,teasp,teconv)
    #### Polar Backlash Plot data ################
    maxlash=round(LASH['Backlash MM'].max(), 3)
    minlash=round(LASH['Backlash MM'].min(), 3)
    avglash=LASH['Backlash MM'].mean()
    avglash=round(avglash, 3)
    laxlim=round((maxlash+.025), 2)
    if laxlim < Lmax+.025:
        laxlim = round((Lmax+.05), 1)
    else:
        laxlim=round(laxlim, 2)
    #### Raw backlash parameters without gain and offset, not currently used #########
    rmaxlash=round(LASH['RAW Backlash MM'].max(), 3)
    rminlash=round(LASH['RAW Backlash MM'].min(), 3)
    ravglash=LASH['RAW Backlash MM'].mean()
    ravglash=round(ravglash, 3)
    rlaxlim=round((rmaxlash+.025), 2)
    if rlaxlim < Lmax+.025:
        rlaxlim = round((Lmax+.05), 1)
    else:
        rlaxlim=round(rlaxlim, 2)
    ravglash=LASH['RAW Backlash MM'].mean()
    ravglash=round(ravglash, 3)
    ########## Backlash Linear Plot #############
    X=(LASH['DEG'])
    Y3=(LASH['Backlash MM'])
    Y4=LASH['PCWrpm']
    Y5=LASH['PCCWrpm']
    avgrpm=LASH['PCWrpm'].mean()
    HEIGHT = 800
    WIDTH = 1280
    global dline_image
    root = tk.Tk()
    root.title("AAM DYNAMIC BACKLASH")

    Axletype = tk.StringVar()
    Axletype.set(Atype)
    PartID = tk.StringVar()
    PartID.set(sernum)
    ratio = Decimal(ratio)
    Axleratio = tk.StringVar()
    Axleratio.set(round(ratio, 3))
    rdia = Decimal(rdia)
    Axledia = tk.StringVar()
    Axledia.set(round(rdia, 0))
    lgain = Decimal(lgain)
    Axlegain = tk.DoubleVar(value=0.000)
    Axlegain.set(round(lgain, 3))
    calcdia = tk.StringVar()
    calcdia.set(round(dia,1))
    offset = Decimal(offset)
    calcset = tk.DoubleVar()
    calcset.set(round(offset, 3))
    gearenc = tk.IntVar()
    gearenc.set(gencts)
    pinenc = tk.IntVar()
    pinenc.set(pencts)
    ratioCW = Decimal(ratioCW)
    fwdratio = tk.DoubleVar()
    fwdratio.set(round(ratioCW, 6))
    ratioCCW = Decimal(ratioCCW)
    revratio = tk.DoubleVar()
    revratio.set(round(ratioCCW, 6))
    avglash = Decimal(avglash)
    lash = tk.StringVar()
    lash.set(round(avglash, 3))
    maxlash = Decimal(maxlash)
    lashmax = tk.StringVar()
    lashmax.set(round(maxlash, 3))
    minlash = Decimal(minlash)
    lashmin = tk.StringVar()
    lashmin.set(round(minlash, 3))
    ########################### POLAR BCAKLASH PLOT ############
    canvas = tk.Canvas(root, height=HEIGHT, width=WIDTH)
    canvas.pack()
    frameb = tk.Frame(root, bg='#E0E0E0', bd=3)
    frameb.place(relx=0, rely=0, relwidth=1, relheight=1)
    dline_image = tk.PhotoImage(file=os.getcwd()+r'\aam lash2\AAMdln.png')
    head_label = tk.Label(root, image=dline_image)
    head_label.pack()
    head_label.place(x=10, y=3)
    frame = tk.Frame(root, bg='#000000', bd=3)
    frame.pack()
    frame.place(relx=0.4, rely=0.0, relwidth=0.6, relheight=0.9)
    figp = plt.Figure(figsize=(6,6), dpi = 100)
    polar = figp.add_subplot(111, projection='polar')
    figp.patch.set_facecolor('#F0F0F0')
    Ang=(LASH['RAD'])
    Y3=(LASH['Backlash MM'])
    polar.set_ylim(0,(laxlim))
    polar.set_rticks([round(laxlim/4, 3), round(laxlim/2, 3), round(laxlim*3/4, 3), round(laxlim,3)])
    polar.plot(Ang,Y3, color='#cc0000', linewidth=4)
    polar.grid(True)
    polar.set_facecolor('#FFFFFF')
    canvas = FigureCanvasTkAgg(figp, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)
    ############## DATA LABELS ##################################################
    cdat = tk.Frame(root, bg='#000000', bd=5)
    cdat.pack()
    cdat.place(x=5, y=100, relwidth=0.35, relheight=0.4)
    lab1 = tk.Label(cdat, text="AXLE TYPE:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab1.place(x=0, y=0, relheight=0.125, relwidth=0.4)
    var1 = tk.Label(cdat, textvariable=Axletype, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var1.place(relx=0.4, y=0, relheight=0.125, relwidth=0.6)
    lab2 = tk.Label(cdat, text="AXLE RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab2.place(x=0, rely=0.125, relheight=0.125, relwidth=0.7)
    var2 = tk.Label(cdat, textvariable=Axleratio, bg='#FFFFFF', justify='center', relief=SUNKEN, font=('arial', 14, 'bold'))
    var2.place(relx=0.7, rely=0.125, relheight=0.125, relwidth=0.3)
    lab3 = tk.Label(cdat, text="GEAR CALCULATING DIA [MM]:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12,'bold'), relief=GROOVE)
    lab3.place(x=0, rely=0.25, relheight=0.125, relwidth=0.7)
    var3 = tk.Label(cdat, textvariable=Axledia, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var3.place(relx=0.7, rely=0.25, relheight=0.125, relwidth=0.3)
    lab4 = tk.Label(cdat, text="BACKLASH GAIN:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab4.place(x=0, rely=0.375, relheight=0.125, relwidth=0.7)
    var4 = tk.Label(cdat, textvariable=Axlegain, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var4.place(relx=0.7, rely=0.375, relheight=0.125, relwidth=0.3)
    lab5 = tk.Label(cdat, text="BACKLASH OFFSET [MM]:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab5.place(x=0, rely=0.5, relheight=0.125, relwidth=0.7)
    var5 = tk.Label(cdat, textvariable=calcset, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var5.place(relx=0.7, rely=0.5, relheight=0.125, relwidth=0.3)
    labeq = tk.Label(cdat, text="REPORTED LASH = (CALCULATED BACKLASH * GAIN) + OFFSET", bg='#F0F0F0', justify='center', font=('arial', 9, 'italic', 'bold'), relief=RAISED)
    labeq.place(x=0, rely=0.625, relheight=0.125, relwidth=1)
    lab6 = tk.Label(cdat, text="GEAR ENCODER (cts/rev):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab6.place(x=0, rely=0.75, relheight=0.125, relwidth=0.7)
    var6 = tk.Label(cdat, textvariable=gearenc, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var6.place(relx=0.7, rely=0.75, relheight=0.125, relwidth=0.3)
    lab7 = tk.Label(cdat, text="PINION ENCODER (cts/rev):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12,'bold'), relief=GROOVE)
    lab7.place(x=0, rely=0.875, relheight=0.125, relwidth=0.7)
    var7 = tk.Label(cdat, textvariable=pinenc, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var7.place(relx=0.7, rely=0.875, relheight=0.125, relwidth=0.3)
    ################## RESULT LABELS ############################################
    cres = tk.Frame(root, bg='#000000', bd=5)
    cres.place(x=5, rely=0.52, relwidth=0.35, relheight=0.375)
    creslab = tk.Label(cres, text="RESULTS", bg='#000000', fg='#F0F0F0', justify='center', font=('arial ', 12, 'bold'))
    creslab.place(x=0, y=0, relheight=.1, relwidth=1)
    lab13 = tk.Label(cres, text="SERIAL NUMBER:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab13.place(x=0, rely=0.1, relheight=0.17, relwidth=0.4)
    var13 = tk.Label(cres, textvariable=PartID, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var13.place(relx=0.4, rely=0.1, relheight=0.17, relwidth=0.6)
    lab8 = tk.Label(cres, text="REPORTED AVG. BACKLASH (MM):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12
    , 'bold'), relief=GROOVE)
    lab8.place(x=0, rely=0.27, relheight=0.17, relwidth=0.7)
    var8 = tk.Label(cres, textvariable=lash, bg='#000000', fg='#FFFFFF', justify='center', font=('arial', 18, 'bold'), relief=SUNKEN)
    var8.place(relx=0.7, rely=0.27, relheight=0.17, relwidth=0.3)
    lab9 = tk.Label(cres, text="MAXIMUM BACKLASH (MM):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab9.place(x=0, rely=0.44, relheight=0.14, relwidth=0.7)
    var9 = tk.Label(cres, textvariable=lashmax, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var9.place(relx=0.7, rely=0.44, relheight=0.14, relwidth=0.3)
    lab10 = tk.Label(cres, text="MINIMUM BACKLASH (MM):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab10.place(x=0, rely=0.58, relheight=0.14, relwidth=0.7)
    var10 = tk.Label(cres, textvariable=lashmin, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var10.place(relx=0.7, rely=0.58, relheight=0.14, relwidth=0.3)
    lab11 = tk.Label(cres, text="MEASURED FORWARD RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab11.place(x=0, rely=0.72, relheight=0.14, relwidth=0.7)
    var11 = tk.Label(cres, textvariable=fwdratio, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var11.place(relx=0.7, rely=0.72, relheight=0.14, relwidth=0.3)
    lab12 = tk.Label(cres, text="MEASURED REVERSE RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab12.place(x=0, rely=0.86, relheight=0.14, relwidth=0.7)
    var12 = tk.Label(cres, textvariable=revratio, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var12.place(relx=0.7, rely=0.86, relheight=0.14, relwidth=0.3)

    def opanalysis():

        global dline_im2

        analysis = Toplevel()
        analysis.title("Data Analysis Graphs")
        PartID = tk.IntVar()
        PartID.set(sernum)
        lash2 = tk.StringVar()
        lash2.set(round(avglash, 3))

        rlash2 = tk.StringVar()
        rlash2.set(round(ravglash, 3))
        ratioCWG = Decimal(ratioCW)
        fwdratio = tk.DoubleVar()
        fwdratio.set(round(ratioCWG, 6))
        ratioCCWG = Decimal(ratioCCW)
        revratio = tk.DoubleVar()
        revratio.set(round(ratioCCWG, 6))

        canvas2 = tk.Canvas(analysis, height=800, width=1280)
        canvas2.pack()
        framebk = tk.Frame(analysis, bg='#F0F0F0', bd=1)
        framebk.place(relx=0, rely=0, relwidth=1, relheight=1)
        framedln = tk.Frame
        dline_im2 = tk.PhotoImage(file=os.getcwd()+r'\aam lash2\AAMdln.png')
        head_label2 = tk.Label(analysis, image=dline_im2)
        head_label2.place(x=5, y=3)
        frame2 = tk.Frame(analysis, bg='#000000', bd=1)
        frame2.pack()
        frame2.place(relx=0.0, rely=0.475, relwidth=0.5, relheight=0.5)
        figlin = plt.Figure(figsize=(4.25,3.25), dpi = 100)
        linear = figlin.add_subplot()
        figlin.patch.set_facecolor('#F0F0F0')
        linear.set(xlabel='DEGREES RING GEAR', ylabel='BACKLASH (MM)',title='NORMAL BACKLASH')
        X=(LASH['DEG'])
        Y3=(LASH['Backlash MM'])
        linear.set_ylim(0,(laxlim))
        linear.set_xlim([0,360])
        linear.grid(True)
        linear.set_xticks([0,45,90,135,180,225,270,315,360])
        linear.set_yticks([round(laxlim/4, 3),round(laxlim/2, 3),round(laxlim*3/4, 3),round(laxlim, 3)])
        linear.set_aspect(round(180/laxlim, 0))
        linear.plot(X,Y3, color='#cc0000', linewidth=4)
        linear.set_facecolor('#FFFFFF')
        canvas2 = FigureCanvasTkAgg(figlin, master=frame2)
        canvas2.draw()
        canvas2.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)
        frame3 = tk.Frame(analysis, bg='#000000', bd=1)
        frame3.pack()
        frame3.place(relx=0.25, rely=0.0, relwidth=0.75, relheight=0.475)
        figTE = plt.Figure(figsize=(7.25,4.5), dpi = 100)
        terror = figTE.add_subplot()

        terror2 = terror.twinx()
        figTE.patch.set_facecolor('#F0F0F0')
        terror.set(xlabel='DEGREES RING GEAR',
        ylabel='TE microradians',
        title='CALCULATED LONG TERM TRANSMISSION ERROR')

        terror2.set(ylabel='Equivalent Backlash TE (MM)')

        X=(LASH['DEG'])
        Y=(LASH['CW TE microrad'])
        Y2=(LASH['CCW TE microrad'])
        terror.set_xlim([0,360])

        #limitsy = [teminl, temaxl]
        #terror.set_ylim(limitsy)

        #print(limitsy)

        terror.set_ylim((teminl),(temaxl))

        terror2.set_ylim(terror.get_ylim())
        secaxis = mticker.FuncFormatter(lambda X, pos: '{:.2f}'.format(teconv*X))
        terror2.yaxis.set_major_formatter(secaxis)


        terror.plot(X, Y, label="FWD TE", color='#cc0000', linewidth=2.5)
        terror.plot(X, Y2, label="REV TE", color='#000000', linewidth=2.5)
        terror.set_xticks([0,45,90,135,180,225,270,315,360])
        terror.yaxis.set_major_locator(mticker.MultipleLocator(1000))
        #terror.yaxis.set_major_locator(mticker.LinearLocator(teindx))
        #terror2.yaxis.set_major_locator(mticker.MultipleLocator(1000))
        #terror2.yaxis.set_major_locator(mticker.LinearLocator(teindx))

        for label in (terror.get_yticklabels()):
            label.set_fontsize(8)
        for label in (terror2.get_yticklabels()):
            label.set_fontsize(8)


        terror.grid()

        terror.legend(loc='best')
        terror.set_aspect(abs(teasp))
        terror.set_facecolor('#FFFFFF')
        canvas3 = FigureCanvasTkAgg(figTE, master=frame3)
        canvas3.draw()
        canvas3.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)
        ##################################################
        frame4 = tk.Frame(analysis, bg='#000000', bd=1)
        frame4.pack()
        frame4.place(relx=0.50, rely=0.475, relwidth=0.5, relheight=0.25)
        figvelF = plt.Figure(figsize=(4.25,1.625), dpi = 100)
        velF = figvelF.add_subplot()
        figvelF.patch.set_facecolor('#F0F0F0')
        figvelF.suptitle('PINION VELOCITY PROFILE DURING TEST')
        velF.set(ylabel='FORWARD (rpm)')
        #axv1.set_ylabel('FORWARD (rpm)', fontsize=11)
        velF.set_ylim([round(0.96*avgrpm, 0), round(1.04*avgrpm, 0)])
        velF.set_yticks([round(0.96*avgrpm, 0), round(avgrpm, 0), round(1.04*avgrpm, 0)])
        #velF.set_yticklabels([round(0.96*avgrpm, 0), round(avgrpm, 0), round(1.04*avgrpm, 0)], fontsize=10)
        velF.grid(True)
        velF.plot(X,Y4, color='#cc0000', linewidth=2.5)
        velF.set_xlim([0,360])
        velF.set_xticks([0,45,90,135,180,225,270,315,360])
        velF.set_facecolor('#FFFFFF')
        velF.set_aspect(round(100/(.08*avgrpm), 0))
        canvas4 = FigureCanvasTkAgg(figvelF, master=frame4)
        canvas4.draw()
        canvas4.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)

        frame5 = tk.Frame(analysis, bg='#000000', bd=1)
        frame5.pack()
        frame5.place(relx=0.50, rely=0.725, relwidth=0.5, relheight=0.25)
        figvelR = plt.Figure(figsize=(4.25,1.625), dpi = 100)
        velR = figvelR.add_subplot()
        figvelR.patch.set_facecolor('#F0F0F0')
        velR.set(ylabel='REVERSE (rpm)')
        velR.set_ylim([round(0.96*avgrpm, 0), round(1.04*avgrpm, 0)])
        velR.set_yticks([round(0.96*avgrpm, 0), round(avgrpm, 0), round(1.04*avgrpm, 0)])
        velR.grid(True)
        velR.plot(X,Y5, color='#000000', linewidth=2.5)
        velR.set_xlim([0,360])
        velR.set_xticks([0,45,90,135,180,225,270,315,360])
        velR.set_facecolor('#FFFFFF')
        velR.set_aspect(round(100/(.08*avgrpm), 0))

        canvas5 = FigureCanvasTkAgg(figvelR, master=frame5)
        canvas5.draw()
        canvas5.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)
        ############## DATA AND BUTTONS #########
        cdat2 = tk.Frame(analysis, bg='#000000', bd=5)
        cdat2.place(x=5, y=100, relwidth=0.24, relheight=0.35)
        lab21 = tk.Label(cdat2, text="PART ID:", bg='#F0F0F0', justify='right', anchor='e', font=('arial', 10, 'bold'), relief=GROOVE)
        lab21.place(x=0, y=0, relheight=0.15, relwidth=0.4)
        var21 = tk.Label(cdat2, textvariable=PartID, bg='#FFFFFF', fg='#000000', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
        var21.place(relx=0.4, y=0, relheight=0.15, relwidth=0.6)
        lab22 = tk.Label(cdat2, text="REPORTED AVG LASH:", bg='#F0F0F0', justify='right', anchor='e', font=('arial', 10, 'bold'), relief=GROOVE)
        lab22.place(relx=0, rely=0.15, relheight=0.15, relwidth=0.6)
        var22 = tk.Label(cdat2, textvariable=lash2, bg='#000000', fg='#F0F0F0', justify='center', font=('arial', 14, 'bold'),relief=SUNKEN)
        var22.place(relx=0.6, rely=0.15, relheight=0.15, relwidth=0.4)

        lab25 = tk.Label(cdat2, text="CALCULATED BACKLASH:", bg='#F0F0F0', justify='right', anchor='e', font=('arial', 10, 'bold'), relief=GROOVE)
        lab25.place(relx=0, rely=0.3, relheight=0.15, relwidth=0.6)
        var25 = tk.Label(cdat2, textvariable=rlash2, bg='#000000', fg='#F0F0F0', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
        var25.place(relx=0.6, rely=0.3, relheight=0.15, relwidth=0.4)
        lab23 = tk.Label(cdat2, text="FORWARD RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
        lab23.place(x=0, rely=0.45, relheight=0.15, relwidth=0.6)
        var23 = tk.Label(cdat2, textvariable=fwdratio, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
        var23.place(relx=0.6, rely=0.45, relheight=0.15, relwidth=0.4)
        lab24 = tk.Label(cdat2, text="REVERSE RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
        lab24.place(x=0, rely=0.60, relheight=0.15, relwidth=0.6)
        var24 = tk.Label(cdat2, textvariable=revratio, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
        var24.place(relx=0.6, rely=0.60, relheight=0.15, relwidth=0.4)

        navpage2 = Button(cdat2, text="RETURN TO\nMAIN PAGE", bg='#cc0000', fg='#FFFFFF', font=('arial', 14, 'bold'),
        relief=RAISED, bd=4, command=analysis.destroy)
        navpage2.place(relx=0.25,rely=0.77, relheight=0.23, relwidth=0.50)
    def getdata():
        global testit
        cruncher = filedialog.askopenfilename(initialdir='/aam lash/test data', title="Backlash Data Files",filetypes=(("data sets", "*.txt"), ("all files", "*.*")))
        testit=cruncher
        datalab = Label(root, text=cruncher)
        datalab.place(x=5, rely=0.95, height=10, width = 400) 
    navpage = tk.Button(root, text="SHOW ANALYSIS\nGRAPHS", bg='#cc0000', fg='#FFFFFF', font=('arial', 12, 'bold'),
    relief=RAISED, bd=4, command=opanalysis)
    navpage.place(relx=.82, rely=0.9, relheight=0.08, relwidth=0.125)
    getdata = tk.Button(root, text="GET NEW\nPART DATA", bg='#cc0000', fg='#FFFFFF', font=('arial', 12, 'bold'),
    relief=RAISED, bd=4, command=getdata)
    getdata.place(relx=0.46, rely=0.9, relheight=0.08, relwidth=0.125)
    def recalc():
        root.destroy()
        #analysis.destroy()
        main()

    logdata = tk.Button(root, text="ANALYZE NEW\nPART DATA", bg='#cc0000', fg='#FFFFFF', font=('arial', 12, 'bold'),
    relief=RAISED, bd=4, command=recalc)
    logdata.place(relx=.64, rely=0.9, relheight=0.08, relwidth=0.125)
    root.mainloop()
 



In [14]:
main()

             CWG          PIN        CCWG         CCWP
0     111.203290   379.962771  111.421880   379.964880
1     111.294992   380.275700  111.511695   380.271049
2     111.386712   380.588671  111.601520   380.577222
3     111.478449   380.901676  111.691354   380.883399
4     111.570202   381.214706  111.781195   381.189583
...          ...          ...         ...          ...
3993  470.735446  1607.908626  470.870207  1607.808213
3994  470.825697  1608.216587  470.960190  1608.115183
3995  470.915947  1608.524566  471.050180  1608.422186
3996  471.006195  1608.832570  471.140177  1608.729224
3997  471.096440  1609.140598  471.230182  1609.036300

[3998 rows x 4 columns]
-200.0 5300.0 -1000.0 6000.0 8 0.02 0.000105


TclError: image "pyimage3" doesn't exist

In [38]:
Enc

,0,1,2,3
0,1.581260e+09,10.5 Line GMC Plant 2,0.980,-0.025
1,1.200000e+01,41.0,36000.000,36000.000
2,5.000000e-04,105.0,0.125,0.250


In [39]:
LASH

,CWG,PIN,CCWG,CCWP,DEG,RAD,TECW,CW TE microrad,TECCW,CCW TE microrad,PINCOMP,GEARCOMP,COMPGRCCW,RAW Backlash MM,Backlash MM,PCWrpm,PCCWrpm
0,111.203290,379.962771,111.421880,379.964880,0.000000,0.000000,0.000000,0.000000,0.217973,3804.347598,-0.002109,-0.000617,111.421263,0.199728,0.170734,NaN,NaN
1,111.294992,380.275700,111.511695,380.271049,0.091702,0.001600,0.000113,1.968453,0.218177,3807.913740,0.004651,0.001361,111.513057,0.199812,0.170816,104.309692,102.056
2,111.386712,380.588671,111.601520,380.577222,0.183422,0.003201,0.000231,4.038599,0.218391,3811.637384,0.011449,0.003351,111.604871,0.199899,0.170901,104.323771,102.058
3,111.478449,380.901676,111.691354,380.883399,0.275159,0.004802,0.000358,6.240423,0.218612,3815.498734,0.018277,0.005349,111.696704,0.199986,0.170986,104.334920,102.059
4,111.570202,381.214706,111.781195,381.189583,0.366912,0.006404,0.000492,8.583121,0.218838,3819.437718,0.025123,0.007353,111.788548,0.200070,0.171068,104.343384,102.061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,470.735446,1607.908626,470.870207,1607.808213,359.532156,6.275020,0.133369,2327.728962,0.297519,5192.684237,0.100412,0.029389,470.899596,0.150410,0.122402,102.646118,102.314
3994,470.825697,1608.216587,470.960190,1608.115183,359.622406,6.276595,0.133485,2329.748599,0.297657,5195.096879,0.101404,0.029679,470.989869,0.150431,0.122422,102.653768,102.323
3995,470.915947,1608.524566,471.050180,1608.422186,359.712657,6.278170,0.133595,2331.665919,0.297792,5197.454595,0.102380,0.029965,471.080144,0.150454,0.122445,102.659749,102.334
3996,471.006195,1608.832570,471.140177,1608.729224,359.802905,6.279745,0.133695,2333.426204,0.297924,5199.763049,0.103346,0.030248,471.170424,0.150483,0.122473,102.668050,102.346


In [ ]:
LASH = pd.read_csv(str(testit), header=4)
LASH.columns = ['CWG', 'PIN', 'CCWG', 'CCWP']
Enc = pd.read_csv(str(testit), header=None,skiprows=1,nrows=3)
encdt=Enc.at[2,0] #time increment of data in seconds
tpin=Enc.at[1,0] #Pinion Teeth
tgear=Enc.at[1,1] #Gear Teeth
lgain =Enc.at[0,2] #diameter gain for lash calcs
offset=Enc.at[0,3] #calculation offset
Lmin=Enc.at[2,2] #Minimum Backlash limit MM
Lmax=Enc.at[2,3] #Maximum Backlash limit MM
Atype=Enc.at[0,1]
sernum=Enc.at[0,0]
sernum=int(sernum)
########## Converting variables to floating decimals ######

tpin=float(tpin) 
tgear=float(tgear) 
ratio=tgear/tpin
rdia=Enc.at[2,1] #raw diameter from file
rdia = float(rdia) 
lgain=float(lgain)
dia=rdia*lgain #calculating diameter
gencts=Enc.at[1,2] #gear encoder counts per rev
gencts = int(gencts)
pencts=Enc.at[1,3] #pinion encoder cts per rev
pencts = int(pencts)
#print(testit)
print(LASH)
gstart=LASH.loc[0,'CWG']
gend=LASH['CWG'][LASH.index[-1]]
pref=LASH.loc[0,'PIN']
pref2=LASH.loc[0,'CCWP']
gref2=LASH.loc[0,'CCWG']
plast=LASH['PIN'][LASH.index[-1]]
plast2=LASH['CCWP'][LASH.index[-1]]
glast2=LASH['CCWG'][LASH.index[-1]]
ratioCW=(plast-pref)/(gend-gstart)
ratioCCW=(plast2-pref2)/(glast2-gref2)

### Starting CCW TE calculation at a matching pinion to illustrate backlash when plotting ##########

glast2C=glast2+(plast-plast2)/ratio
CWteend=gend-(plast-pref)/ratio-gstart

LASH['DEG']=(LASH['CWG']-gstart)
LASH['RAD']=(LASH['DEG']*np.pi/180)
#print(gstart,gend)
#print(pref,pref2,gref2,plast,plast2,glast2,glast2C)
#print(ratioCW,ratioCCW)
#print(LASH)
#### Gear TE in Counts Clockwise & Counterclockwise ######## 
LASH['TECW']=(LASH['CWG']-((LASH['PIN']-pref)/(tgear/tpin)+gstart))
LASH['CW TE microrad']=(LASH['TECW']*np.pi*1000000/180)
LASH['TECCW']=(LASH['CCWG']-(glast2C-(plast-LASH['CCWP'])/(tgear/tpin))+(glast2C-gend)+CWteend)
LASH['CCW TE microrad']=(LASH['TECCW']*np.pi*1000000/180)
#print(LASH)
### "Raw" Backlash Calculations without gain and offset ##########
LASH['PINCOMP']=(LASH['PIN']-LASH['CCWP'])
LASH['GEARCOMP']=(LASH['PINCOMP']/(tgear/tpin))
LASH['COMPGRCCW']=(LASH['GEARCOMP']+LASH['CCWG'])
LASH['RAW Backlash MM']=((LASH['COMPGRCCW']-LASH['CWG'])/360*np.pi*rdia)
#print(LASH)
### Backlash calculations including gain and offset in values #########
LASH['Backlash MM']=((LASH['COMPGRCCW']-LASH['CWG'])/360*np.pi*dia+offset)
#print(LASH)
##### Pinion velocity calculations based on time increment of data acquisition #######
for i in range(1, len(LASH)):
    LASH.loc[i, 'PCWrpm'] = (LASH.loc[i, 'PIN'] - LASH.loc[i-1, 'PIN'])/360/encdt*60
for i in range(1, len(LASH)):
    LASH.loc[i, 'PCCWrpm'] = (LASH.loc[i, 'CCWP'] - LASH.loc[i-1, 'CCWP'])/360/encdt*60
LASH.loc[LASH['PCCWrpm']>150,'PCCWrpm']= 0.0
LASH.loc[LASH['PCCWrpm']<10,'PCCWrpm']= 'NaN'
#print(LASH)
#LASH.to_csv('LASH-DATA.csv', sep=',')
#### Parameters calculated to facilitate TE PLot limits and scaling of the plot ###########
temaxcw=LASH['CW TE microrad'].max()
temincw=LASH['CW TE microrad'].min()
temaxccw=LASH['CCW TE microrad'].max()
teminccw=LASH['CCW TE microrad'].min()
if temaxcw > temaxccw:
    temax = temaxcw+100
else:
    temax= temaxccw+100
if temincw < teminccw:
    temin = temincw-100
else:
    temin= teminccw-100
teasp=(130/abs((temax-temin))) 

temax=round(temax, -2)
temin=round(temin, -2)
teasp=abs(round(teasp, 2))

def round_up(n):
    multiplier = .001
    return math.ceil(n*multiplier)/multiplier

def round_dwn(n):
    multiplier = 0.001
    return math.floor(n*multiplier)/multiplier


temaxl=round_up(temax)
teminl=round_dwn(temin)
teindx=int((temaxl-teminl)/1000+1)

teconv=rdia/1000000.0

print(temin,temax,teminl,temaxl,teindx,teasp,teconv)
#### Polar Backlash Plot data ################
maxlash=round(LASH['Backlash MM'].max(), 3)
minlash=round(LASH['Backlash MM'].min(), 3)
avglash=LASH['Backlash MM'].mean()
avglash=round(avglash, 3)
laxlim=round((maxlash+.025), 2)
if laxlim < Lmax+.025:
    laxlim = round((Lmax+.05), 1)
else:
    laxlim=round(laxlim, 2)
#### Raw backlash parameters without gain and offset, not currently used #########
rmaxlash=round(LASH['RAW Backlash MM'].max(), 3)
rminlash=round(LASH['RAW Backlash MM'].min(), 3)
ravglash=LASH['RAW Backlash MM'].mean()
ravglash=round(ravglash, 3)
rlaxlim=round((rmaxlash+.025), 2)
if rlaxlim < Lmax+.025:
    rlaxlim = round((Lmax+.05), 1)
else:
    rlaxlim=round(rlaxlim, 2)
ravglash=LASH['RAW Backlash MM'].mean()
ravglash=round(ravglash, 3)
########## Backlash Linear Plot #############
X=(LASH['DEG'])
Y3=(LASH['Backlash MM'])
Y4=LASH['PCWrpm']
Y5=LASH['PCCWrpm']
avgrpm=LASH['PCWrpm'].mean()
HEIGHT = 800
WIDTH = 1280
global dline_image
root = tk.Tk()
root.title("AAM DYNAMIC BACKLASH")

Axletype = tk.StringVar()
Axletype.set(Atype)
PartID = tk.StringVar()
PartID.set(sernum)
ratio = Decimal(ratio)
Axleratio = tk.StringVar()
Axleratio.set(round(ratio, 3))
rdia = Decimal(rdia)
Axledia = tk.StringVar()
Axledia.set(round(rdia, 0))
lgain = Decimal(lgain)
Axlegain = tk.DoubleVar(value=0.000)
Axlegain.set(round(lgain, 3))
calcdia = tk.StringVar()
calcdia.set(round(dia,1))
offset = Decimal(offset)
calcset = tk.DoubleVar()
calcset.set(round(offset, 3))
gearenc = tk.IntVar()
gearenc.set(gencts)
pinenc = tk.IntVar()
pinenc.set(pencts)
ratioCW = Decimal(ratioCW)
fwdratio = tk.DoubleVar()
fwdratio.set(round(ratioCW, 6))
ratioCCW = Decimal(ratioCCW)
revratio = tk.DoubleVar()
revratio.set(round(ratioCCW, 6))
avglash = Decimal(avglash)
lash = tk.StringVar()
lash.set(round(avglash, 3))
maxlash = Decimal(maxlash)
lashmax = tk.StringVar()
lashmax.set(round(maxlash, 3))
minlash = Decimal(minlash)
lashmin = tk.StringVar()
lashmin.set(round(minlash, 3))
########################### POLAR BCAKLASH PLOT ############
canvas = tk.Canvas(root, height=HEIGHT, width=WIDTH)
canvas.pack()
frameb = tk.Frame(root, bg='#E0E0E0', bd=3)
frameb.place(relx=0, rely=0, relwidth=1, relheight=1)
dline_image = tk.PhotoImage(file=os.getcwd()+r'\aam lash2\AAMdln.png')
print(1)
# try:
head_label = tk.Label(root, image=dline_image)
head_label.pack()
head_label.place(x=10, y=3)
print(2)
# except: 
#     print("not 2")
frame = tk.Frame(root, bg='#000000', bd=3)
frame.pack()
frame.place(relx=0.4, rely=0.0, relwidth=0.6, relheight=0.9)
figp = plt.Figure(figsize=(6,6), dpi = 100)
polar = figp.add_subplot(111, projection='polar')
figp.patch.set_facecolor('#F0F0F0')
Ang=(LASH['RAD'])
Y3=(LASH['Backlash MM'])
polar.set_ylim(0,(laxlim))
polar.set_rticks([round(laxlim/4, 3), round(laxlim/2, 3), round(laxlim*3/4, 3), round(laxlim,3)])
polar.plot(Ang,Y3, color='#cc0000', linewidth=4)
polar.grid(True)
polar.set_facecolor('#FFFFFF')
canvas = FigureCanvasTkAgg(figp, master=frame)
canvas.draw()
canvas.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)
############## DATA LABELS ##################################################
cdat = tk.Frame(root, bg='#000000', bd=5)
cdat.pack()
cdat.place(x=5, y=100, relwidth=0.35, relheight=0.4)
lab1 = tk.Label(cdat, text="AXLE TYPE:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab1.place(x=0, y=0, relheight=0.125, relwidth=0.4)
var1 = tk.Label(cdat, textvariable=Axletype, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var1.place(relx=0.4, y=0, relheight=0.125, relwidth=0.6)
lab2 = tk.Label(cdat, text="AXLE RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab2.place(x=0, rely=0.125, relheight=0.125, relwidth=0.7)
var2 = tk.Label(cdat, textvariable=Axleratio, bg='#FFFFFF', justify='center', relief=SUNKEN, font=('arial', 14, 'bold'))
var2.place(relx=0.7, rely=0.125, relheight=0.125, relwidth=0.3)
lab3 = tk.Label(cdat, text="GEAR CALCULATING DIA [MM]:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12,'bold'), relief=GROOVE)
lab3.place(x=0, rely=0.25, relheight=0.125, relwidth=0.7)
var3 = tk.Label(cdat, textvariable=Axledia, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var3.place(relx=0.7, rely=0.25, relheight=0.125, relwidth=0.3)
lab4 = tk.Label(cdat, text="BACKLASH GAIN:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab4.place(x=0, rely=0.375, relheight=0.125, relwidth=0.7)
var4 = tk.Label(cdat, textvariable=Axlegain, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var4.place(relx=0.7, rely=0.375, relheight=0.125, relwidth=0.3)
lab5 = tk.Label(cdat, text="BACKLASH OFFSET [MM]:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab5.place(x=0, rely=0.5, relheight=0.125, relwidth=0.7)
var5 = tk.Label(cdat, textvariable=calcset, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var5.place(relx=0.7, rely=0.5, relheight=0.125, relwidth=0.3)
labeq = tk.Label(cdat, text="REPORTED LASH = (CALCULATED BACKLASH * GAIN) + OFFSET", bg='#F0F0F0', justify='center', font=('arial', 9, 'italic', 'bold'), relief=RAISED)
labeq.place(x=0, rely=0.625, relheight=0.125, relwidth=1)
lab6 = tk.Label(cdat, text="GEAR ENCODER (cts/rev):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab6.place(x=0, rely=0.75, relheight=0.125, relwidth=0.7)
var6 = tk.Label(cdat, textvariable=gearenc, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var6.place(relx=0.7, rely=0.75, relheight=0.125, relwidth=0.3)
lab7 = tk.Label(cdat, text="PINION ENCODER (cts/rev):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12,'bold'), relief=GROOVE)
lab7.place(x=0, rely=0.875, relheight=0.125, relwidth=0.7)
var7 = tk.Label(cdat, textvariable=pinenc, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var7.place(relx=0.7, rely=0.875, relheight=0.125, relwidth=0.3)
################## RESULT LABELS ############################################
cres = tk.Frame(root, bg='#000000', bd=5)
cres.place(x=5, rely=0.52, relwidth=0.35, relheight=0.375)
creslab = tk.Label(cres, text="RESULTS", bg='#000000', fg='#F0F0F0', justify='center', font=('arial ', 12, 'bold'))
creslab.place(x=0, y=0, relheight=.1, relwidth=1)
lab13 = tk.Label(cres, text="SERIAL NUMBER:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab13.place(x=0, rely=0.1, relheight=0.17, relwidth=0.4)
var13 = tk.Label(cres, textvariable=PartID, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var13.place(relx=0.4, rely=0.1, relheight=0.17, relwidth=0.6)
lab8 = tk.Label(cres, text="REPORTED AVG. BACKLASH (MM):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12
, 'bold'), relief=GROOVE)
lab8.place(x=0, rely=0.27, relheight=0.17, relwidth=0.7)
var8 = tk.Label(cres, textvariable=lash, bg='#000000', fg='#FFFFFF', justify='center', font=('arial', 18, 'bold'), relief=SUNKEN)
var8.place(relx=0.7, rely=0.27, relheight=0.17, relwidth=0.3)
lab9 = tk.Label(cres, text="MAXIMUM BACKLASH (MM):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab9.place(x=0, rely=0.44, relheight=0.14, relwidth=0.7)
var9 = tk.Label(cres, textvariable=lashmax, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var9.place(relx=0.7, rely=0.44, relheight=0.14, relwidth=0.3)
lab10 = tk.Label(cres, text="MINIMUM BACKLASH (MM):", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab10.place(x=0, rely=0.58, relheight=0.14, relwidth=0.7)
var10 = tk.Label(cres, textvariable=lashmin, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var10.place(relx=0.7, rely=0.58, relheight=0.14, relwidth=0.3)
lab11 = tk.Label(cres, text="MEASURED FORWARD RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab11.place(x=0, rely=0.72, relheight=0.14, relwidth=0.7)
print(3)
var11 = tk.Label(cres, textvariable=fwdratio, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var11.place(relx=0.7, rely=0.72, relheight=0.14, relwidth=0.3)
lab12 = tk.Label(cres, text="MEASURED REVERSE RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
lab12.place(x=0, rely=0.86, relheight=0.14, relwidth=0.7)
var12 = tk.Label(cres, textvariable=revratio, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
var12.place(relx=0.7, rely=0.86, relheight=0.14, relwidth=0.3)

def opanalysis():

    global dline_im2

    analysis = Toplevel()
    analysis.title("Data Analysis Graphs")
    PartID = tk.IntVar()
    PartID.set(sernum)
    lash2 = tk.StringVar()
    lash2.set(round(avglash, 3))

    rlash2 = tk.StringVar()
    rlash2.set(round(ravglash, 3))
    ratioCWG = Decimal(ratioCW)
    fwdratio = tk.DoubleVar()
    fwdratio.set(round(ratioCWG, 6))
    ratioCCWG = Decimal(ratioCCW)
    revratio = tk.DoubleVar()
    revratio.set(round(ratioCCWG, 6))

    canvas2 = tk.Canvas(analysis, height=800, width=1280)
    canvas2.pack()
    print(4)
    framebk = tk.Frame(analysis, bg='#F0F0F0', bd=1)
    framebk.place(relx=0, rely=0, relwidth=1, relheight=1)
    framedln = tk.Frame
    dline_im2 = tk.PhotoImage(file=os.getcwd()+r'\aam lash2\AAMdln.png')
    head_label2 = tk.Label(analysis, image=dline_im2)
    head_label2.place(x=5, y=3)
    frame2 = tk.Frame(analysis, bg='#000000', bd=1)
    frame2.pack()
    frame2.place(relx=0.0, rely=0.475, relwidth=0.5, relheight=0.5)
    figlin = plt.Figure(figsize=(4.25,3.25), dpi = 100)
    linear = figlin.add_subplot()
    figlin.patch.set_facecolor('#F0F0F0')
    linear.set(xlabel='DEGREES RING GEAR', ylabel='BACKLASH (MM)',title='NORMAL BACKLASH')
    X=(LASH['DEG'])
    Y3=(LASH['Backlash MM'])
    linear.set_ylim(0,(laxlim))
    linear.set_xlim([0,360])
    linear.grid(True)
    linear.set_xticks([0,45,90,135,180,225,270,315,360])
    linear.set_yticks([round(laxlim/4, 3),round(laxlim/2, 3),round(laxlim*3/4, 3),round(laxlim, 3)])
    linear.set_aspect(round(180/laxlim, 0))
    linear.plot(X,Y3, color='#cc0000', linewidth=4)
    linear.set_facecolor('#FFFFFF')
    canvas2 = FigureCanvasTkAgg(figlin, master=frame2)
    canvas2.draw()
    canvas2.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)
    frame3 = tk.Frame(analysis, bg='#000000', bd=1)
    frame3.pack()
    frame3.place(relx=0.25, rely=0.0, relwidth=0.75, relheight=0.475)
    figTE = plt.Figure(figsize=(7.25,4.5), dpi = 100)
    terror = figTE.add_subplot()

    terror2 = terror.twinx()
    figTE.patch.set_facecolor('#F0F0F0')
    terror.set(xlabel='DEGREES RING GEAR',
    ylabel='TE microradians',
    title='CALCULATED LONG TERM TRANSMISSION ERROR')

    terror2.set(ylabel='Equivalent Backlash TE (MM)')

    X=(LASH['DEG'])
    Y=(LASH['CW TE microrad'])
    Y2=(LASH['CCW TE microrad'])
    terror.set_xlim([0,360])
    print(5)

    #limitsy = [teminl, temaxl]
    #terror.set_ylim(limitsy)

    #print(limitsy)

    terror.set_ylim((teminl),(temaxl))

    terror2.set_ylim(terror.get_ylim())
    secaxis = mticker.FuncFormatter(lambda X, pos: '{:.2f}'.format(teconv*X))
    terror2.yaxis.set_major_formatter(secaxis)


    terror.plot(X, Y, label="FWD TE", color='#cc0000', linewidth=2.5)
    terror.plot(X, Y2, label="REV TE", color='#000000', linewidth=2.5)
    terror.set_xticks([0,45,90,135,180,225,270,315,360])
    terror.yaxis.set_major_locator(mticker.MultipleLocator(1000))
    #terror.yaxis.set_major_locator(mticker.LinearLocator(teindx))
    #terror2.yaxis.set_major_locator(mticker.MultipleLocator(1000))
    #terror2.yaxis.set_major_locator(mticker.LinearLocator(teindx))

    for label in (terror.get_yticklabels()):
        label.set_fontsize(8)
    for label in (terror2.get_yticklabels()):
        label.set_fontsize(8)


    terror.grid()

    terror.legend(loc='best')
    terror.set_aspect(abs(teasp))
    terror.set_facecolor('#FFFFFF')
    canvas3 = FigureCanvasTkAgg(figTE, master=frame3)
    canvas3.draw()
    canvas3.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)
    ##################################################
    frame4 = tk.Frame(analysis, bg='#000000', bd=1)
    frame4.pack()
    frame4.place(relx=0.50, rely=0.475, relwidth=0.5, relheight=0.25)
    figvelF = plt.Figure(figsize=(4.25,1.625), dpi = 100)
    velF = figvelF.add_subplot()
    figvelF.patch.set_facecolor('#F0F0F0')
    figvelF.suptitle('PINION VELOCITY PROFILE DURING TEST')
    velF.set(ylabel='FORWARD (rpm)')
    #axv1.set_ylabel('FORWARD (rpm)', fontsize=11)
    velF.set_ylim([round(0.96*avgrpm, 0), round(1.04*avgrpm, 0)])
    velF.set_yticks([round(0.96*avgrpm, 0), round(avgrpm, 0), round(1.04*avgrpm, 0)])
    #velF.set_yticklabels([round(0.96*avgrpm, 0), round(avgrpm, 0), round(1.04*avgrpm, 0)], fontsize=10)
    velF.grid(True)
    velF.plot(X,Y4, color='#cc0000', linewidth=2.5)
    velF.set_xlim([0,360])
    print(6)
    velF.set_xticks([0,45,90,135,180,225,270,315,360])
    velF.set_facecolor('#FFFFFF')
    velF.set_aspect(round(100/(.08*avgrpm), 0))
    canvas4 = FigureCanvasTkAgg(figvelF, master=frame4)
    canvas4.draw()
    canvas4.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)

    frame5 = tk.Frame(analysis, bg='#000000', bd=1)
    frame5.pack()
    frame5.place(relx=0.50, rely=0.725, relwidth=0.5, relheight=0.25)
    figvelR = plt.Figure(figsize=(4.25,1.625), dpi = 100)
    velR = figvelR.add_subplot()
    figvelR.patch.set_facecolor('#F0F0F0')
    velR.set(ylabel='REVERSE (rpm)')
    velR.set_ylim([round(0.96*avgrpm, 0), round(1.04*avgrpm, 0)])
    velR.set_yticks([round(0.96*avgrpm, 0), round(avgrpm, 0), round(1.04*avgrpm, 0)])
    velR.grid(True)
    velR.plot(X,Y5, color='#000000', linewidth=2.5)
    velR.set_xlim([0,360])
    velR.set_xticks([0,45,90,135,180,225,270,315,360])
    velR.set_facecolor('#FFFFFF')
    velR.set_aspect(round(100/(.08*avgrpm), 0))

    canvas5 = FigureCanvasTkAgg(figvelR, master=frame5)
    canvas5.draw()
    canvas5.get_tk_widget().pack(side = TOP, fill = BOTH, expand = True)
    ############## DATA AND BUTTONS #########
    cdat2 = tk.Frame(analysis, bg='#000000', bd=5)
    cdat2.place(x=5, y=100, relwidth=0.24, relheight=0.35)
    lab21 = tk.Label(cdat2, text="PART ID:", bg='#F0F0F0', justify='right', anchor='e', font=('arial', 10, 'bold'), relief=GROOVE)
    lab21.place(x=0, y=0, relheight=0.15, relwidth=0.4)
    var21 = tk.Label(cdat2, textvariable=PartID, bg='#FFFFFF', fg='#000000', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var21.place(relx=0.4, y=0, relheight=0.15, relwidth=0.6)
    lab22 = tk.Label(cdat2, text="REPORTED AVG LASH:", bg='#F0F0F0', justify='right', anchor='e', font=('arial', 10, 'bold'), relief=GROOVE)
    lab22.place(relx=0, rely=0.15, relheight=0.15, relwidth=0.6)
    var22 = tk.Label(cdat2, textvariable=lash2, bg='#000000', fg='#F0F0F0', justify='center', font=('arial', 14, 'bold'),relief=SUNKEN)
    var22.place(relx=0.6, rely=0.15, relheight=0.15, relwidth=0.4)

    lab25 = tk.Label(cdat2, text="CALCULATED BACKLASH:", bg='#F0F0F0', justify='right', anchor='e', font=('arial', 10, 'bold'), relief=GROOVE)
    lab25.place(relx=0, rely=0.3, relheight=0.15, relwidth=0.6)
    var25 = tk.Label(cdat2, textvariable=rlash2, bg='#000000', fg='#F0F0F0', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var25.place(relx=0.6, rely=0.3, relheight=0.15, relwidth=0.4)
    lab23 = tk.Label(cdat2, text="FORWARD RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab23.place(x=0, rely=0.45, relheight=0.15, relwidth=0.6)
    var23 = tk.Label(cdat2, textvariable=fwdratio, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var23.place(relx=0.6, rely=0.45, relheight=0.15, relwidth=0.4)
    lab24 = tk.Label(cdat2, text="REVERSE RATIO:", bg='#F0F0F0', justify='right', anchor='e', font=('Courier', 12, 'bold'), relief=GROOVE)
    lab24.place(x=0, rely=0.60, relheight=0.15, relwidth=0.6)
    var24 = tk.Label(cdat2, textvariable=revratio, bg='#FFFFFF', justify='center', font=('arial', 14, 'bold'), relief=SUNKEN)
    var24.place(relx=0.6, rely=0.60, relheight=0.15, relwidth=0.4)

    navpage2 = Button(cdat2, text="RETURN TO\nMAIN PAGE", bg='#cc0000', fg='#FFFFFF', font=('arial', 14, 'bold'),
    relief=RAISED, bd=4, command=analysis.destroy)
    print(7)
    navpage2.place(relx=0.25,rely=0.77, relheight=0.23, relwidth=0.50)
def getdata():
    global testit
    cruncher = filedialog.askopenfilename(initialdir='/aam lash/test data', title="Backlash Data Files",filetypes=(("data sets", "*.txt"), ("all files", "*.*")))
    testit=cruncher
    datalab = Label(root, text=cruncher)
    datalab.place(x=5, rely=0.95, height=10, width = 400) 
navpage = tk.Button(root, text="SHOW ANALYSIS\nGRAPHS", bg='#cc0000', fg='#FFFFFF', font=('arial', 12, 'bold'),
relief=RAISED, bd=4, command=opanalysis)
navpage.place(relx=.82, rely=0.9, relheight=0.08, relwidth=0.125)
getdata = tk.Button(root, text="GET NEW\nPART DATA", bg='#cc0000', fg='#FFFFFF', font=('arial', 12, 'bold'),
relief=RAISED, bd=4, command=getdata)
getdata.place(relx=0.46, rely=0.9, relheight=0.08, relwidth=0.125)
def recalc():
    root.destroy()
    #analysis.destroy()
    main()

logdata = tk.Button(root, text="ANALYZE NEW\nPART DATA", bg='#cc0000', fg='#FFFFFF', font=('arial', 12, 'bold'),
relief=RAISED, bd=4, command=recalc)
print(8)
logdata.place(relx=.64, rely=0.9, relheight=0.08, relwidth=0.125)
root.mainloop()




             CWG          PIN        CCWG         CCWP
0     111.203290   379.962771  111.421880   379.964880
1     111.294992   380.275700  111.511695   380.271049
2     111.386712   380.588671  111.601520   380.577222
3     111.478449   380.901676  111.691354   380.883399
4     111.570202   381.214706  111.781195   381.189583
...          ...          ...         ...          ...
3993  470.735446  1607.908626  470.870207  1607.808213
3994  470.825697  1608.216587  470.960190  1608.115183
3995  470.915947  1608.524566  471.050180  1608.422186
3996  471.006195  1608.832570  471.140177  1608.729224
3997  471.096440  1609.140598  471.230182  1609.036300

[3998 rows x 4 columns]
-200.0 5300.0 -1000.0 6000.0 8 0.02 0.000105
1
2
3
8
             CWG          PIN        CCWG         CCWP
0     111.203290   379.962771  111.421880   379.964880
1     111.294992   380.275700  111.511695   380.271049
2     111.386712   380.588671  111.601520   380.577222
3     111.478449   380.901676  111.691354 

In [22]:
canvas3 

NameError: name 'canvas3' is not defined